In [ ]:
import cv2
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO('best.pt')

# 동영상 파일 경로
video_path = 'video.mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

detected_labels = set()  # 중복 없이 탐지된 라벨을 저장할 집합(set)

while True:
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # 객체 탐지 수행
    results = model.predict(frame, conf=0.5)

    # 예측 결과를 프레임에 그리기 및 집합에 라벨 추가
    for result in results:
        for box in result.boxes:
            # 텐서를 개별 값으로 변환
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            label = result.names[int(box.cls[0])]

            # 바운딩 박스와 레이블 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # 탐지된 라벨 집합에 추가 (중복 방지)
            detected_labels.add(label)

    # 결과 프레임을 화면에 표시
    cv2.imshow('Video YOLO Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()

# 탐지된 라벨 출력
print("Detected Labels:", detected_labels)